In [1]:
!pip install deepr[cpu]

Looking in indexes: http://build-nexus.prod.crto.in/repository/pypi/simple


# Hyper Parameter Tuning

This notebook builds upon the previous [pipeline example](https://criteo.github.io/deepr/getting_started/pipeline.html). The goal is to perform hyperparameter search by varying the learning rate and batch size.

To launch an HP search, the steps are

1. Download / retrieve some config and its macros (usually saved as MLFlow artifacts).
2. Prepare the config by adding new macro parameters for a macro named "params" (for example change `"learning_rate": 0.1` to `"learning_rate": "$params:learning_rate"`).
3. Define a sampler.
4. Launch a tuning job using the prepared config, macros and sampler.

First, some imports

In [2]:
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger("cluster_pack").setLevel(logging.CRITICAL)

In [3]:
import numpy as np
import deepr as dpr

## Prepare config

You can either manually create a config or download it from other experiments (using mlflow)

To download it from mlflow you can either use bash:


```bash
deepr download_config_and_macros_from_mlflow --run_id=841d3f9f2ba4b69921b426a81fd --tracking_uri=$MLFLOW_TRACKING_URI
```

It will download config and macros artifact to `config.json` and `macros.json` paths from some run id

Alternatively, you can download it from python:
    
```python
run_id="841d3f9f2ba4b69921b426a81fd"
tracking_uri="https://mlflow.crto.in/"

dpr.cli.download_config_and_macros_from_mlflow(
    run_id=run_id,
    path_config="config.json",
    path_macros="macros.json",
    tracking_uri=tracking_uri
)
```

To download macros you can do:

```bash
deepr add_macro --config=config.json --macros=macros.json --params=learning_rate,batch_size
```

Or with python

```python
dpr.cli.add_macro(
    config="config.json",
    macros="macros.json",
    params=["learning_rate", "batch_size"],
)
```

You can then use:

```python
# Read job config and macros
job = dpr.io.read_json("config.json")
macros = dpr.io.read_json("macros.json")
```

Alternatively, define macros and configs manually:

In [4]:
job = {
    "type": "deepr.jobs.yarn_launcher.YarnLauncher",
    "job": {
        "type": "deepr.jobs.Pipeline",
        "eval": None,
        "jobs": [
            {
                "type": "deepr.examples.multiply.jobs.build.Build",
                "path_dataset": "viewfs://root/user/deepr/dev/example/data.tfrecord",
                "num_examples": 1000,
            },
            {
                "type": "deepr.jobs.YarnTrainer",
                "trainer": {
                    "type": "deepr.jobs.Trainer",
                    "path_model": "$paths:path_model",  # Uses macro "paths:path_model"
                    "pred_fn": {"type": "deepr.examples.multiply.layers.model.Multiply"},
                    "loss_fn": {"type": "deepr.examples.multiply.layers.loss.SquaredL2"},
                    "optimizer_fn": {
                        "type": "deepr.optimizers.TensorflowOptimizer",
                        "optimizer": "Adam",
                        "learning_rate": "$params:learning_rate",  # Uses macro "params:learning_rate"
                    },
                    "train_spec": {
                        "type": "deepr.jobs.TrainSpec",
                        "max_steps": 1000
                    },
                    "prepro_fn": {
                        "type": "deepr.examples.multiply.prepros.default.DefaultPrepro",  # Uses macro "params:batch_size"
                        "batch_size": "$params:batch_size",
                        "repeat_size": None,
                    },
                    "train_input_fn": {
                        "type": "deepr.readers.TFRecordReader",
                        "path": "viewfs://root/user/deepr/dev/example/data.tfrecord",
                        "num_parallel_reads": 8,
                        "num_parallel_calls": 8,
                        "shuffle": True,
                    },
                    "eval_input_fn": {
                        "type": "deepr.readers.TFRecordReader",
                        "path": "viewfs://root/user/deepr/dev/example/data.tfrecord",
                        "num_parallel_reads": 8,
                        "num_parallel_calls": 8,
                        "shuffle": True,
                    },
                },
                "config": {
                    "type": "deepr.jobs.YarnTrainerConfig"
                },
            },
        ],
    },
    "config": {
        "type": "deepr.jobs.YarnLauncherConfig",
        "path_pex_prefix": "viewfs://root/user/deepr/dev/example/envs",
        "cpu_ignored_packages": ["thx"]
    },
}

We also used a dynamic macro to set the path to the model dynamically (every experiment needs to use a different model path).

In [5]:
import datetime
import time

class Paths(dict):
    """Macro that generates new path_model based on date."""

    def __init__(self, path_model: str = None, path_dataset: str = None):
        now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        if path_model is None:
            path_model = f"viewfs://root/user/deepr/dev/example/models/{now}"
        if path_dataset is None:
            path_dataset = f"viewfs://root/user/deepr/dev/example/data.tfrecord"
        super().__init__(path_model=path_model, path_dataset=path_dataset)

macros = {
    "paths": {
        "type": "__main__.Paths"
    },
    "params": {
        "learning_rate": 0.01,
        "batch_size": 16
    }
}

## Launch

Now that the config is prepared, we can launch hyper parameter tuning using the `ParamsSampler` job.

The only thing that it does is sampling some learning rate values and use them as macros.

First, let's define a sampler for the parameters

In [6]:
param_grid = {
    "learning_rate": np.logspace(start=-3, stop=-1, num=10),
    "batch_size": [8, 16, 32, 64],
}
sampler = dpr.jobs.ParamsSampler(param_grid, n_iter=5, seed=42)

and use the sampler with our config and macros to launch hyper params tuning with


In [7]:
HAS_HADOOP = False

if HAS_HADOOP:
    tuner = dpr.jobs.ParamsTuner(job=job, macros=macros, sampler=sampler)
    tuner.run()